# Work with Text and Images

You need to run in the terminal:
```bash
pip install matplotlib
```

```bash
pip install -U torch torchvision
pip install -U "transformers>=4.41.0"
pip install -U qwen-vl-utils
pip install -U pillow
```


# Setup

In [ ]:
# add auto reload for notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import io
import shutil
import base64

from conversational_toolkit.chunking.pdf_chunker import PDFChunker

import matplotlib.pyplot as plt

# Parse document and chunk

In [ ]:
chunker = PDFChunker()

In [ ]:
chunks = chunker.make_chunks(
    "../../data/EPD_cardboard_redbox_cartonpallet.pdf",
    write_images=True,
    image_path="../../data/tmp_images/",
)

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {chunk.title} ({chunk.mime_type})")
    if chunk.mime_type.startswith("image/"):
        image_data = base64.b64decode(chunk.content)

        image = plt.imread(io.BytesIO(image_data), format=chunk.mime_type.split("/")[1])

        plt.figure(figsize=(2, 2))
        plt.imshow(image)
        plt.axis("off")

plt.show()

----------------------